# Исследование набора данных о продажах и продуктах интернет-магазина

**Описание проекта**

Имеется набор данных о продажах и продуктах интернет-магазина. Перед нами стоит 15 различных вопросов (ad-hoc запросов), ответить на которые можно, проанализировав эти данные.

**Задачи исследования**

Ответить на вопросы и написать код для решения каждой задачи.

**Ход исследования**

Ссылка на форму блиц-теста:
* https://docs.google.com/forms/d/e/1FAIpQLSfoaggqEAUmL6MK3KWHGeHucqLjEtbu8MbRVtc-hDBgstHbWw/viewform?usp=send_form

Входные данные о продажах и продуктах интернет-магазина доступны по ссылкам:

* 'sales_data.csv'
https://gist.githubusercontent.com/andron23/0675d643fe001b08a66271402a7b609b/raw/231deb6c35c43ca9089b197d94d7d48e093bfb06/sales_data.csv

* 'products_data.csv'
https://gist.githubusercontent.com/andron23/fc2b28de07d5017f951c559f51fcb286/raw/be9f8cd81b28f3c19d314ef908aa68e9f41ae975/products_data.csv

О качестве данных ничего не известно. Поэтому перед тем, как отвечать на вопрсы исследования и делать вывод, понадобится обзор данных.

Проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки, поищем возможность исправить недочеты в данных.

После ответа на поставленные в исследовании вопросы, сделаем общий вывод.

Таким образом, исследование пройдёт в четыре этапа:

[Этап 1. Загрузка и обзор данных](#1)

[1.1. Откроем данные](#2)\
[1.2. Изучим общую информацию](#3)

[Этап 2. Предобработка данных](#4)

[2.1. Пропуски значений](#5)\
[2.2. Дубликаты](#6)\
[2.3. Исследование данных по столбцам](#7)

[Этап 3. Ответы на вопросы исследования](#8)

[3.1. В каком регионе было продано наибольшее количество продуктов категории "Смартфоны"?](#9)\
[3.2. Какова разница между максимальной и минимальной средней ценой продуктов категории "Ноутбуки" между всеми производителями?](#10)\
[3.3. Какой покупатель совершил заказы на наибольшую сумму по всем временам?](#11)\
[3.4. Какова доля продаж продуктов категории "Планшеты" от общей выручки за все время?](#12)\
[3.5. Какова разница в средней цене продуктов категории "Смартфоны" между двумя самыми крупными регионами по продажам?](#13)\
[3.6. Сколько в среднем уникальных категорий продуктов было продано по всем регионам?](#14)\
[3.7. Какой производитель представлен в наибольшем количестве различных категорий продуктов?](#15)\
[3.8. В каком месяце было продано больше всего продуктов категории "Планшеты" в регионе "Север"?](#16)\
[3.9. Какой производитель продал продукты на наибольшую сумму в регионе "Юг" за март 2023 года?](#17)\
[3.10. Сколько покупателей совершили заказы на продукты хотя бы трех разных категорий?](#18)\
[3.11. Какова доля продаж продуктов производителя "C" от общей выручки за все время в регионе "Север"?](#19)\
[3.12. Найдите покупателей, которые купили у более чем 15% уникальных производителей.](#20)\
[3.13. Какое максимальное количество продуктов было продано в один заказ?](#21)\
[3.14. Какой производитель имеет самую высокую среднюю цену продукта среди всех категорий?](#22)\
[3.15. Каково общее количество уникальных производителей, чьи продукты были куплены менее чем в 3 регионах?](#23)

[Этап 4. Общий вывод](#24)

## Этап 1. Загрузка и обзор данных <a id='1'></a>

Составим первое впечатление о продажах и продуктах интернет-магазина.

In [1]:
# Импортируем библиотеки
import pandas as pd
import numpy
from datetime import datetime, timedelta
import requests

import warnings

In [2]:
# Чтобы часть столбцов в дальнейшем не скрывалась, настроим принудительное отображение
pd.set_option('display.max_columns', None)
# Настроим ширину столбцов
pd.set_option('display.max_colwidth', 999)
# Настроим отображение чисел с точностью до двух знаков после запятой (до сотых)
# 3.514284e+05 — научный формат вывода чисел. Означает 3.51 * 105
# pd.set_option('display.float_format', '{:.2f}'.format)
# pd.set_option.display.float_format = '{:.2f}'.format

In [3]:
warnings.simplefilter(action='ignore', category=Warning)

### 1.1. Откроем данные <a id='2'></a>

In [4]:
# Прочитаем файлы 'sales_data.csv' и 'products_data.csv' из папки и сохраним их в переменных 'sales' и 'products'
# path = r'C:\Users'
# sales = pd.read_csv(r'sales_data.csv', encoding='1251')
# products = pd.read_csv(r'products_data.csv', encoding='1251')

Воспользуемся библиотекой **requests** (она предоставляет простой в использовании интерфейс для отправки HTTP-запросов и обработки ответов).\
Для скачивания файла можно использовать метод **get()**, который отправляет GET-запрос на указанный URL и возвращает ответ сервера.\
Запишем полученные данные в файл с помощью стандартных средств Python.

In [5]:
# Скачаем по URL файл 'sales_data.csv'
url = 'https://gist.githubusercontent.com/andron23/0675d643fe001b08a66271402a7b609b/raw/231deb6c35c43ca9089b197d94d7d48e093bfb06/sales_data.csv'
response = requests.get(url)

with open('sales_data.csv', 'wb') as file:
    file.write(response.content)

# Прочитаем файл 'sales_data.csv' и сохраним его в переменной 'sales_data' и выведем первые 3 строки датафрейма
sales_data = pd.read_csv('sales_data.csv')
sales_data.head(3)

,order_id,product_id,customer_id,order_date,quantity,price_per_unit,total_price,payment_method,region
0,1,101,1001,2022-02-15,2,500,1000,Карта,Север
1,2,102,1002,2022-03-20,1,800,800,Наличные,Юг
2,3,103,1003,2022-03-25,3,300,900,Карта,Запад


In [6]:
# Скачаем по URL файл 'products_data.csv'
url = 'https://gist.githubusercontent.com/andron23/fc2b28de07d5017f951c559f51fcb286/raw/be9f8cd81b28f3c19d314ef908aa68e9f41ae975/products_data.csv'
response = requests.get(url)

with open('products_data.csv', 'wb') as file:
    file.write(response.content)

# Прочитаем файл 'products_data.csv', сохраним его в переменной 'products_data' и выведем первые 3 строки датафрейма
products_data = pd.read_csv('products_data.csv')
products_data.head(3)

,product_id,product_name,category,manufacturer
0,101,Ноутбук HP Pavilion 15,Ноутбуки,A
1,102,Смартфон Samsung Galaxy S21,Смартфоны,B
2,103,Планшет Apple iPad Air,Планшеты,C


### 1.2. Изучим общую информацию <a id='3'></a>

In [7]:
# Выведем основную информацию о датафрейме 'sales_data'
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        40 non-null     int64 
 1   product_id      40 non-null     int64 
 2   customer_id     40 non-null     int64 
 3   order_date      40 non-null     object
 4   quantity        40 non-null     int64 
 5   price_per_unit  40 non-null     int64 
 6   total_price     40 non-null     int64 
 7   payment_method  40 non-null     object
 8   region          40 non-null     object
dtypes: int64(6), object(3)
memory usage: 2.9+ KB


In [8]:
# Выведем основную информацию о датафрейме 'products_data'
products_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_id    30 non-null     int64 
 1   product_name  30 non-null     object
 2   category      30 non-null     object
 3   manufacturer  30 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.1+ KB


In [9]:
# Посмотрим описательные статистики датасета 'sales_data'
sales_data.describe()

,order_id,product_id,customer_id,quantity,price_per_unit,total_price
count,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000
mean,20.500000,120.500000,1007.325000,1.825000,505.625000,811.250000
std,11.690452,11.690452,4.763039,0.957762,245.046274,339.586381
min,1.000000,101.000000,1001.000000,1.000000,150.000000,150.000000
25%,10.750000,110.750000,1003.000000,1.000000,300.000000,600.000000
50%,20.500000,120.500000,1007.000000,2.000000,450.000000,800.000000
75%,30.250000,130.250000,1010.250000,2.000000,700.000000,1000.000000
max,40.000000,140.000000,1018.000000,4.000000,1100.000000,1500.000000


In [10]:
# Посмотрим описательные статистики датасета 'products_data'
products_data.describe()

,product_id
count,30.000000
mean,115.500000
std,8.803408
min,101.000000
25%,108.250000
50%,115.500000
75%,122.750000
max,130.000000


**Вывод**

На первый взгляд данные в хорошем состоянии.\
Продолжим дальнейший анализ.

## Этап 2. Предобработка данных <a id='4'></a>

Поищем пропуски значений и дубликаты, исследуем данные по столбцам.

### 2.1. Пропуски значений <a id='5'></a>

In [11]:
# Посчитаем, сколько в таблице 'sales_data' пропущенных значений
sales_data.isna().sum()

order_id          0
product_id        0
customer_id       0
order_date        0
quantity          0
price_per_unit    0
total_price       0
payment_method    0
region            0
dtype: int64

In [12]:
# Посчитаем, сколько в таблице 'products_data' пропущенных значений
products_data.isna().sum()

product_id      0
product_name    0
category        0
manufacturer    0
dtype: int64

Обзор недостающих значений показывает, что пропусков нет.

### 2.2. Дубликаты <a id='6'></a>

In [13]:
# Посчитаем явные дубликаты в таблице 'sales_data'
sales_data.duplicated().sum()

0

In [14]:
# Посчитаем явные дубликаты в таблице 'products_data'
products_data.duplicated().sum()

0

Дубликаты отсутствуют.

### 2.3. Исследование данных по столбцам <a id='7'></a>

In [15]:
# Выведем первые 3 строки датафрейма 'sales_data'
sales_data.head(3)

,order_id,product_id,customer_id,order_date,quantity,price_per_unit,total_price,payment_method,region
0,1,101,1001,2022-02-15,2,500,1000,Карта,Север
1,2,102,1002,2022-03-20,1,800,800,Наличные,Юг
2,3,103,1003,2022-03-25,3,300,900,Карта,Запад


In [16]:
# Преобразуем данные в столбце с датой из значений типа object в datetime
sales_data['order_date'] = pd.to_datetime(sales_data['order_date'])

In [17]:
# Проверим правильность выполнения преобразования
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        40 non-null     int64         
 1   product_id      40 non-null     int64         
 2   customer_id     40 non-null     int64         
 3   order_date      40 non-null     datetime64[ns]
 4   quantity        40 non-null     int64         
 5   price_per_unit  40 non-null     int64         
 6   total_price     40 non-null     int64         
 7   payment_method  40 non-null     object        
 8   region          40 non-null     object        
dtypes: datetime64[ns](1), int64(6), object(2)
memory usage: 2.9+ KB


In [18]:
# Посмотрим на уникальные значения 'quantity', посчитаем количество таких строк
sales_data['quantity'].value_counts()

1    19
2    12
3     6
4     3
Name: quantity, dtype: int64

In [19]:
# Посмотрим на уникальные значения 'price_per_unit', посчитаем количество таких строк
sales_data['price_per_unit'].value_counts()

300     6
450     3
400     3
500     3
600     3
700     3
200     3
800     2
750     2
250     2
350     2
275     1
150     1
1100    1
900     1
550     1
1000    1
650     1
950     1
Name: price_per_unit, dtype: int64

In [20]:
# Посмотрим на уникальные значения 'total_price', посчитаем количество таких строк
sales_data['total_price'].value_counts()

800     5
900     4
1000    4
700     4
1200    3
1350    2
600     2
1100    2
300     2
750     1
250     1
1400    1
950     1
500     1
650     1
550     1
200     1
1500    1
150     1
400     1
450     1
Name: total_price, dtype: int64

In [21]:
# Посмотрим на уникальные значения способа платежа 'payment_method', посчитаем количество таких строк
sales_data['payment_method'].value_counts()

Карта       27
Наличные    13
Name: payment_method, dtype: int64

In [22]:
# Посмотрим на уникальные значения региона 'region', посчитаем количество таких строк
sales_data['region'].value_counts()

Север     11
Юг        11
Восток     9
Запад      9
Name: region, dtype: int64

In [23]:
# Выведем первые 3 строки датафрейма 'products_data'
products_data.head(3)

,product_id,product_name,category,manufacturer
0,101,Ноутбук HP Pavilion 15,Ноутбуки,A
1,102,Смартфон Samsung Galaxy S21,Смартфоны,B
2,103,Планшет Apple iPad Air,Планшеты,C


In [24]:
# Посмотрим на уникальные значения наименования продукта 'product_name', посчитаем количество таких строк
products_data['product_name'].value_counts()

Ноутбук HP Pavilion 15                1
Телевизор Samsung The Frame           1
Смартфон Oppo Find X3 Pro             1
Телевизор LG OLED CX                  1
Телевизор Sony Bravia X900H           1
Ноутбук Dell XPS 15                   1
Наушники JBL Tune 750BTNC             1
Планшет Amazon Fire HD 10             1
Наушники Sennheiser HD 660S           1
Наушники Sony WH-1000XM4              1
Ноутбук Lenovo ThinkPad X1 Carbon     1
Наушники Beats Solo Pro               1
Планшет Lenovo Tab P11 Pro            1
Планшет Apple iPad Air                1
Смартфон Google Pixel 6               1
Наушники AKG K712 Pro                 1
Ноутбук ASUS ROG Zephyrus G14         1
Ноутбук Microsoft Surface Laptop 4    1
Телевизор Sony A8H OLED               1
Планшет Lenovo Yoga Tab 13            1
Смартфон OnePlus 9 Pro                1
Планшет Huawei MatePad Pro            1
Телевизор LG NanoCell NANO90          1
Смартфон iPhone 13 Pro                1
Ноутбук HP Spectre x360               1


In [25]:
# Посмотрим на уникальные значения категории товара 'category', посчитаем количество таких строк
products_data['category'].value_counts()

Планшеты      6
Смартфоны     6
Телевизоры    6
Ноутбуки      6
Наушники      6
Name: category, dtype: int64

In [26]:
# Посмотрим на уникальные значения 'manufacturer', посчитаем количество таких строк
products_data['manufacturer'].value_counts()

A    6
B    6
E    6
C    6
D    6
Name: manufacturer, dtype: int64

**Вывод**

* Данные находятся в хорошем состоянии — пропущенные значения и дубликаты отсутствуют;
* Был проведен обзор данных по столбцам на предмет уникальности значений и количества таких строк.

## Этап 3. Ответы на вопросы исследования <a id='8'></a>

Объединим таблицы 'sales' и 'products'.

In [27]:
# Сравним длины датафреймов 'sales_data' и 'products_data'
print(len(sales_data))
print(len(products_data))

40
30


In [28]:
# Выведем уникальные значения 'product_id' датафреймов 'sales_data' и 'products_data'
print(sales_data['product_id'].unique())
print(products_data['product_id'].unique())

[101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136
 137 138 139 140]
[101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130]


In [29]:
# Объединим датафреймы 'sales_data' и 'products_data'
merged_data = sales_data.merge(products_data, on='product_id')
merged_data

,order_id,product_id,customer_id,order_date,quantity,price_per_unit,total_price,payment_method,region,product_name,category,manufacturer
0,1,101,1001,2022-02-15,2,500,1000,Карта,Север,Ноутбук HP Pavilion 15,Ноутбуки,A
1,2,102,1002,2022-03-20,1,800,800,Наличные,Юг,Смартфон Samsung Galaxy S21,Смартфоны,B
2,3,103,1003,2022-03-25,3,300,900,Карта,Запад,Планшет Apple iPad Air,Планшеты,C
3,4,104,1001,2022-04-10,2,400,800,Карта,Восток,Наушники Sony WH-1000XM4,Наушники,D
4,5,105,1004,2022-04-15,1,700,700,Наличные,Север,Телевизор LG OLED CX,Телевизоры,E
5,6,106,1005,2022-05-05,2,600,1200,Карта,Юг,Ноутбук Lenovo ThinkPad X1 Carbon,Ноутбуки,A
6,7,107,1003,2022-05-20,1,200,200,Наличные,Запад,Смартфон iPhone 13 Pro,Смартфоны,C
7,8,108,1006,2022-06-01,3,450,1350,Карта,Восток,Планшет Samsung Galaxy Tab S7,Планшеты,B
8,9,109,1007,2022-06-10,1,1000,1000,Карта,Север,Наушники Bose QuietComfort 35 II,Наушники,E
9,10,110,1004,2022-06-15,2,350,700,Наличные,Юг,Телевизор Sony Bravia X900H,Телевизоры,D


### 3.1. В каком регионе было продано наибольшее количество продуктов категории "Смартфоны"? <a id='9'></a>

In [30]:
region_with_most_smartphone_sales = (
    merged_data.
    query('category  == "Смартфоны"').
    groupby('region')['quantity'].
    sum().
    idxmax()
)
region_with_most_smartphone_sales

'Восток'

**Ответ:** 
* Наибольшее количество продуктов категории "Смартфоны" было продано в регионе 'Восток'.

### 3.2. Какова разница между максимальной и минимальной средней ценой продуктов категории "Ноутбуки" между всеми производителями? <a id='10'></a>

 Ответ округлите до целых.

In [80]:
manufacturer_avg_price = (
    merged_data.
    query('category == "Ноутбуки"').
    groupby('manufacturer')['price_per_unit'].
    mean()
)
round(max(manufacturer_avg_price) - min(manufacturer_avg_price))

200

**Ответ:**
* Разница между макс. и мин. средней ценой продуктов категории "Ноутбуки" между всеми производителями равна 200 у.е..

### 3.3. Какой покупатель совершил заказы на наибольшую сумму по всем временам? <a id='11'></a>

In [108]:
top_spending_customer = (
    sales_data.
    groupby('customer_id')['total_price'].
    sum().
    idxmax()
)
top_spending_customer

1003

**Ответ:**
* Покупатель c customer_id = 1003 совершил заказы на наибольшую сумму по всем временам.

### 3.4. Какова доля продаж продуктов категории "Планшеты" от общей выручки за все время? <a id='12'></a>

Ответ округлите до 2 знака после запятой. Разделитель разряда — точка.

In [82]:
tablets_revenue_ratio = (
    merged_data.
    query('category == "Планшеты"')['total_price'].
    sum() / merged_data['total_price'].
    sum()
)
round(tablets_revenue_ratio, 2)

0.25

**Ответ:**
* Доля продаж продуктов категории "Планшеты" от общей выручки за все время равна 0.25.

### 3.5. Какова разница в средней цене продуктов категории "Смартфоны" между двумя самыми крупными регионами по продажам? <a id='13'></a>

Ответ округлите до целых.

In [83]:
top_2_regions = (
    merged_data.
    groupby('region')['total_price'].
    sum().
    nlargest(2)
)
top_2_regions

region
Восток    7100
Юг        7000
Name: total_price, dtype: int64

In [84]:
top_regions_smartphones = (
    merged_data[(merged_data['region'].
                 isin(top_2_regions.index)) & (merged_data['category'] == 'Смартфоны')]
)
top_regions_smartphones

,order_id,product_id,customer_id,order_date,quantity,price_per_unit,total_price,payment_method,region,product_name,category,manufacturer
1,2,102,1002,2022-03-20,1,800,800,Наличные,Юг,Смартфон Samsung Galaxy S21,Смартфоны,B
11,12,112,1009,2022-07-15,4,275,1100,Карта,Восток,Смартфон Xiaomi Mi 11,Смартфоны,D
21,22,122,1006,2022-12-02,2,300,600,Наличные,Юг,Смартфон OnePlus 9 Pro,Смартфоны,D


In [85]:
price_difference = (
    top_regions_smartphones.
    groupby('region')['price_per_unit'].
    mean().
    diff().
    abs().
    iloc[-1]
)
price_difference

275.0

**Ответ:**
* Разница в средней цене продуктов категории "Смартфоны" между двумя самыми крупными регионами по продажам (Востоком и Югом) равна 275.

### 3.6. Сколько в среднем уникальных категорий продуктов было продано по всем регионам? <a id='14'></a>

In [86]:
unique_categories_per_region = (
    merged_data.
    groupby('region')['category'].
    nunique().
    mean()
)
unique_categories_per_region

5.0

**Ответ:**
* В среднем по всем регионам было продано 5 уникальных категорий продуктов.

### 3.7. Какой производитель представлен в наибольшем количестве различных категорий продуктов? <a id='15'></a>

In [38]:
manufacturer_with_most_unique_categories = (
    merged_data.
    groupby('manufacturer')['category'].
    nunique().
    idxmax()
)
manufacturer_with_most_unique_categories

'E'

**Ответ:**
* Производитель 'E' представлен в наибольшем количестве различных категорий продуктов.

### 3.8. В каком месяце было продано больше всего продуктов категории "Планшеты" в регионе "Север"? <a id='16'></a>

In [87]:
north_tablets_data = (
    merged_data.
    query('category == "Планшеты" & region == "Север"')
)
north_tablets_data

,order_id,product_id,customer_id,order_date,quantity,price_per_unit,total_price,payment_method,region,product_name,category,manufacturer
12,13,113,1010,2022-08-05,1,900,900,Наличные,Север,Планшет Huawei MatePad Pro,Планшеты,C


In [88]:
north_tablets_data['month'] = (
    north_tablets_data['order_date'].
    dt.
    month
)
north_tablets_data

,order_id,product_id,customer_id,order_date,quantity,price_per_unit,total_price,payment_method,region,product_name,category,manufacturer,month
12,13,113,1010,2022-08-05,1,900,900,Наличные,Север,Планшет Huawei MatePad Pro,Планшеты,C,8


In [89]:
month_with_most_north_tablet_sales = (
    north_tablets_data.
    groupby('month')['quantity'].
    sum().
    idxmax()
)
month_with_most_north_tablet_sales

8

**Ответ:**
* В 8 месяце (августе) было продано больше всего продуктов категории "Планшеты" в регионе "Север".

### 3.9. Какой производитель продал продукты на наибольшую сумму в регионе "Юг" за март 2023 года? <a id='17'></a>

In [90]:
south_march_2023_data = (
    merged_data[(merged_data['region'] == 'Юг') & 
                (merged_data['order_date'].dt.month == 3) & 
                (merged_data['order_date'].dt.year == 2023)]
)
south_march_2023_data

,order_id,product_id,customer_id,order_date,quantity,price_per_unit,total_price,payment_method,region,product_name,category,manufacturer
29,30,130,1018,2023-03-30,2,350,700,Карта,Юг,Телевизор Samsung The Frame,Телевизоры,B


In [91]:
manufacturer_with_highest_revenue = (
    south_march_2023_data.
    groupby('manufacturer')['total_price'].
    sum().
    idxmax()
)
manufacturer_with_highest_revenue

'B'

**Ответ:**
* Производитель 'B' продал продукты на наибольшую сумму в регионе "Юг" за март 2023 года.

### 3.10. Сколько покупателей совершили заказы на продукты хотя бы трех разных категорий? <a id='18'></a>

In [92]:
customer_unique_categories = (
    merged_data.
    groupby('customer_id')['category'].
    nunique()
)
customer_unique_categories

customer_id
1001    2
1002    1
1003    3
1004    1
1005    1
1006    2
1007    2
1008    2
1009    2
1010    2
1011    2
1012    1
1013    1
1014    1
1015    1
1016    1
1017    1
1018    1
Name: category, dtype: int64

In [93]:
customers_with_multiple_categories = (
    customer_unique_categories[customer_unique_categories >= 3].
    count()
)
# num_customers_with_multiple_categories = len(customers_with_multiple_categories)
num_customers_with_multiple_categories

1

**Ответ:**
* 1 покупатель совершил заказы на продукты хотя бы трех разных категорий.

### 3.11. Какова доля продаж продуктов производителя "C" от общей выручки за все время в регионе "Север"? <a id='19'></a>

Ответ округлите до 2 знака после запятой. В качестве разделителя разрядов используйте точку.

In [95]:
revenue_ratio_manufacturer_c_north = (
    merged_data.
    query('manufacturer == "C" & region == "Север"')['total_price'].sum() / 
    merged_data.query('region == "Север"')['total_price'].
    sum()
)
round(revenue_ratio_manufacturer_c_north, 2)

0.27

**Ответ:**
* Доля продаж продуктов производителя "C" от общей выручки за все время в регионе "Север" равна 0.27.

### 3.12. Найдите покупателей, которые купили у более чем 15% уникальных производителей. <a id='20'></a>

Примечание: Уникальными производителями считайте уникальную связку "Категория = Производитель".

Если таких покупателей несколько — напишите их через запятую с пробелом в порядке возрастания. 

In [97]:
total_manufactures = (
    int(merged_data.
        groupby(['category', 'manufacturer'])['manufacturer'].
        agg({'nunique'}).
        sum()
       )
)
total_manufactures

15

In [98]:
unique_manufacturers_per_customer = (
    merged_data.
    groupby(['customer_id', 'category', 'manufacturer'])['manufacturer'].
    agg({'nunique'})
)
unique_manufacturers_per_customer

nunique
customer_id category   manufacturer         
1001        Наушники   D                   1
            Ноутбуки   A                   1
                       C                   1
1002        Смартфоны  B                   1
1003        Наушники   A                   1
            Планшеты   C                   1
            Смартфоны  C                   1
1004        Телевизоры D                   1
                       E                   1
1005        Ноутбуки   A                   1
1006        Планшеты   B                   1
            Смартфоны  D                   1
1007        Наушники   E                   1
            Ноутбуки   C                   1
1008        Ноутбуки   E                   1
            Планшеты   E                   1
1009        Наушники   A                   1
            Смартфоны  D                   1
1010        Планшеты   C                   1
            Телевизоры B                   1
1011        Наушники   A                   1
            Телевизоры B                   1
1012        Ноутбуки   C                   1
1013        Смартфоны  D                   1
1014        Планшеты   E                   1
1015        Телевизоры B                   1
1016        Планшеты   E                   1
1017        Смартфоны  D                   1
1018        Телевизоры B                   1

In [99]:
percent_from_total = (
    unique_manufacturers_per_customer.
    groupby('customer_id').
    sum().
    apply(lambda x: x / total_manufactures).
    reset_index()
)
percent_from_total

,customer_id,nunique
0,1001,0.200000
1,1002,0.066667
2,1003,0.200000
3,1004,0.133333
4,1005,0.066667
5,1006,0.133333
6,1007,0.133333
7,1008,0.133333
8,1009,0.133333
9,1010,0.133333


In [100]:
customers_with_more_than_15 = (
    percent_from_total[percent_from_total['nunique'] > 0.15]['customer_id'].
    tolist()
)
# customers_with_more_than_15['customer_id'].tolist()
customers_with_more_than_15

[1001, 1003]

**Ответ:**
* Покупатели с customer_id = 1001 и customer_id = 1003 купили у более чем 15% уникальных производителей.

### 3.13. Какое максимальное количество продуктов было продано в один заказ? <a id='21'></a>

In [101]:
max_quantity_per_order = (
    sales_data.
    groupby('order_id')['quantity'].
    sum().
    max()
)
max_quantity_per_order

4

**Ответ:**
* Максимальное количество продуктов, которое было продано в одном заказе, равно 4.

### 3.14. Какой производитель имеет самую высокую среднюю цену продукта среди всех категорий? <a id='22'></a>

In [102]:
average_price_per_manufacturer_per_category = (
    merged_data.
    groupby(['manufacturer', 'category'])['price_per_unit'].
    mean()
)
average_price_per_manufacturer_per_category

manufacturer  category  
A             Наушники       462.500000
              Ноутбуки       550.000000
B             Планшеты       450.000000
              Смартфоны      800.000000
              Телевизоры     537.500000
C             Ноутбуки       583.333333
              Планшеты       600.000000
              Смартфоны      200.000000
D             Наушники       400.000000
              Смартфоны      356.250000
              Телевизоры     350.000000
E             Наушники      1000.000000
              Ноутбуки       750.000000
              Планшеты       550.000000
              Телевизоры     700.000000
Name: price_per_unit, dtype: float64

In [103]:
manufacturer_with_highest_average_price = (
    average_price_per_manufacturer_per_category.
    groupby('manufacturer').
    mean().
    idxmax()
)
manufacturer_with_highest_average_price

'E'

**Ответ:**
* Производитель 'E' имеет самую высокую среднюю цену продукта среди всех категорий.

### 3.15. Каково общее количество уникальных производителей, чьи продукты были куплены менее, чем в 3 регионах? <a id='23'></a>

In [104]:
unique_regions_per_manufacturer = (
    merged_data.
    groupby(['manufacturer', 'region'])['region'].
    nunique()
)
unique_regions_per_manufacturer

manufacturer  region
A             Восток    1
              Запад     1
              Север     1
              Юг        1
B             Восток    1
              Запад     1
              Север     1
              Юг        1
C             Восток    1
              Запад     1
              Север     1
              Юг        1
D             Восток    1
              Запад     1
              Север     1
              Юг        1
E             Восток    1
              Запад     1
              Север     1
              Юг        1
Name: region, dtype: int64

In [105]:
manufacturers_with_products_in_multiple_regions = (
    unique_regions_per_manufacturer.
    groupby('manufacturer').
    apply(lambda x: (len(x) <= 2))
)
manufacturers_with_products_in_multiple_regions

manufacturer
A    False
B    False
C    False
D    False
E    False
Name: region, dtype: bool

In [106]:
total_manufacturers_with_products_in_multiple_regions = (
    manufacturers_with_products_in_multiple_regions.
    sum()
)
total_manufacturers_with_products_in_multiple_regions

0

**Ответ:**
* Общее количество уникальных производителей, чьи продукты были куплены менее чем в 3 регионах, равно 0.

## Этап 4. Общий вывод <a id='24'></a>

В ходе исследования данных о продажах и продуктах интернет-магазина мы сделали проверку на наличие некорректных значений — пропусков и дубликатов. Преобразовали данные в нужный формат. Ответили на 15 различных вопросов (ad-hoc запросов). Написали код для решения каждой задачи.

1. Наибольшее количество продуктов категории "Смартфоны" было продано в регионе 'Восток'.
2. Разница между макс. и мин. средней ценой продуктов категории "Ноутбуки" между всеми производителями составляет 200 у.е..
3. Покупатель с customer_id = 1003 совершил заказы на наибольшую сумму по всем временам.
4. Доля продаж продуктов категории "Планшеты" от общей выручки за все время равна 0.25.
5. Разница в средней цене продуктов категории "Смартфоны" между двумя самыми крупными регионами по продажам (Востоком и Югом) равна 275.
6. В среднем по всем регионам было продано 5 уникальных категорий продуктов.
7. Производитель 'E' представлен в наибольшем количестве различных категорий продуктов.
8. В 8 месяце (августе) было продано больше всего продуктов категории "Планшеты" в регионе "Север".
9. Производитель 'B' продал продукты на наибольшую сумму в регионе "Юг" за март 2023 года.
10. 1 покупатель совершил заказы на продукты хотя бы трех разных категорий.
11. Доля продаж продуктов производителя "C" от общей выручки за все время в регионе "Север" равна 0.27.
12. Покупатели с customer_id = 1001 и customer_id = 1003 купили у более чем 15% уникальных производителей.
13. Максимальное количество продуктов, которое было продано в одном заказе, равно 4.
14. Производитель 'E' имеет самую высокую среднюю цену продукта среди всех категорий.
15. Общее количество уникальных производителей, чьи продукты были куплены менее чем в 3 регионах, равно 0.